In [5]:
import pandas as pd
import numpy as np

In [6]:
df=pd.read_excel(r"C:\Users\tangyaoyao\Jupyter\carbon_dot.xlsx")

AttributeError: module 'pandas' has no attribute 'read_excel'

In [3]:
df.head()

,CA质量（g）,DEA体积（ml）,水体积（ml）,微波强度（低火18 解冻36 中火58 中高火81 高火100 ）,微波时间（min）,最佳激发,最佳发射,产率（%）
0,1.0,0.50,20,36,4,380,480,1.660
1,1.0,0.55,20,36,4,320,408,0.762
2,1.0,0.60,20,36,4,320,408,0.434
3,1.0,0.65,20,36,4,320,408,0.923
4,1.0,0.70,20,36,4,320,408,1.500


In [4]:
#将dataframe数据转化为numpy
df_value=df.values

In [5]:
#删除激发和发射两列数
df_value=np.delete(df_value,[5,6],axis=1)

In [6]:
df_value[1,]

array([ 1.   ,  0.55 , 20.   , 36.   ,  4.   ,  0.762])

In [7]:
X=df_value[:,:-1]
Y=df_value[:,-1]

In [8]:
print(X.shape)
print(Y.shape)

(190, 5)
(190,)


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [10]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import numpy as np
def evaluate(y_true,y_pred): 
    r2 = r2_score(y_true,y_pred)   
    mse=mean_squared_error(y_true,y_pred)
    mae=mean_absolute_error(y_true,y_pred)
    pr = np.corrcoef(y_true,y_pred)
    return [mse,mae,r2,pr[0][1]]

In [11]:

#handle warnings
import warnings
warnings.filterwarnings("ignore")

In [12]:
from sklearn.preprocessing import  PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
verbose=False
n_jobs = 4
save_csv = True


# cross validation setup
Ntrials = 6
outter_nsplit = 5

tot_count = Ntrials * outter_nsplit

# Results store
rf_result = []
xgb_result =[]
dt_result = []
mlp_result = []


for i in range(Ntrials):
    print("when i=",i)
    
    kf=KFold(n_splits=5,random_state=i+10,shuffle=True)
    for train_indices,test_indices in kf.split(X):
        x_train,x_test=X[train_indices],X[test_indices]
        y_train,y_test=Y[train_indices],Y[test_indices]
        
         
        #randomforest
        param_grid = {'n_estimators':[int(x) for x in np.linspace(start =50,stop = 150,num = 10)],
             'min_samples_split':[20,30,40,50,60],
             'min_samples_leaf':[3,4,5],
             'max_depth':[4,5,6,7,8]}
       # param_grid={}
        RF = RandomForestRegressor()
        RF_grid = GridSearchCV(RF,param_grid = param_grid,scoring = 'neg_mean_squared_error',cv = 3,n_jobs = n_jobs)
        RF_grid.fit(x_train,y_train)
        y_pred=RF_grid.predict(x_test)
      # print("y_pred.shape",y_pred.shape)
       # print("y_test.shape",y_test.shape)
        error=evaluate(y_test,y_pred)
        #print("error:",error)
        rf_result.append(error)
        
        # xgboost
        param_grid = dict(objective=[ 'reg:squarederror'],
                            learning_rate=[0.01],
                            n_estimators=[300,500,700], #100,,300,400,500
                            colsample_bylevel = [0.5,0.7,0.9],
                          gamma=[0,0.2], #0,0.1,0.2,0.3,0.4
                          max_depth =[3,7,11], # [3,7,11]]
                          reg_lambda = [0.1,1,10], #[0.1,1,10]
                         # reg_alpha = [1],
                           subsample=[0.4,0.7,1])

        xgb = XGBRegressor()

        xgb_grid = GridSearchCV(xgb,param_grid = param_grid,scoring='r2',verbose=verbose,n_jobs=n_jobs)
        xgb_grid.fit(x_train, y_train)
        y_pred = xgb_grid.predict(x_test)
    
        error=evaluate(y_test,y_pred)
        xgb_result.append(error)
        
        
        #Decision tree
        param_prid = { 'max_depth':[30,50,60,100],
                       'min_impurity_decrease':[0.1,0.2,0.5]}
        dt=DecisionTreeRegressor()
        dt_grid = GridSearchCV(dt,param_grid=param_prid,n_jobs=n_jobs,verbose=verbose)
        dt_grid.fit(x_train,y_train)
        y_pred=dt_grid.predict(x_test)
        error=evaluate(y_test,y_pred)
        dt_result.append(error)

        # MLP
        mlp_clf = Pipeline([            
                ('sc', StandardScaler()), 
                ('reg',  MLPRegressor())
                ])
        tuned_parameters = dict(reg__hidden_layer_sizes=[[5],[10],[20],[5,5],[10,10],[20,20],[5,5,5],[10,10,10],[20,20,20]],
                          reg__alpha=[1e-4,1e-3, 1e-2, 1e-1,1], #L2 penalty (regularization term) parameter.
                          reg__early_stopping=[True],
                         reg__solver= ['lbfgs'])
        mlp_cv = GridSearchCV(mlp_clf, tuned_parameters,scoring='r2',verbose=verbose,n_jobs=n_jobs)
        mlp_cv.fit(x_train, y_train)
        y_pred = mlp_cv.predict(x_test)
        error = evaluate(y_test,y_pred)
        mlp_result.append(error)



when i= 0
when i= 1
when i= 2
when i= 3
when i= 4
when i= 5


In [13]:
print(rf_result)

[[10.433843019077807, 2.434032047845605, 0.4979686625796331, 0.7098555485686128], [4.996993886670332, 1.7726887127051159, 0.7272286356803623, 0.8991539031191221], [6.992243907786605, 2.092166906201308, 0.4981661673201181, 0.7183966451337606], [11.03773062655804, 2.173671155892373, 0.1399608478340224, 0.4992613555008318], [10.353398968040157, 1.9012538371561114, 0.42853739982465033, 0.6765084457859574], [7.662255589834778, 2.1195123629736643, 0.5175852315738687, 0.7482236634076262], [10.090020424412229, 1.9455187784040242, 0.4088219647390182, 0.6435960438995301], [13.828269187425423, 2.465416807400301, 0.23116099605803697, 0.48631824108229904], [7.962395199788284, 2.222867858311337, 0.4555679372672481, 0.6809520304272738], [7.19507185798027, 2.052790523998651, 0.6124175075566617, 0.8064170466552182], [6.812019473372257, 1.905609726684494, 0.4127655590509546, 0.6468490583790881], [12.375942180556363, 2.3274910275519214, 0.45280092565334995, 0.6833342012959972], [12.075641752176228, 2.416

In [14]:
print(xgb_result)

[[10.577688116781816, 2.32477507746847, 0.49104745946687156, 0.7041365984746812], [4.838545141025949, 1.6779586723478217, 0.7358778919140843, 0.8896204685396548], [6.656132184478927, 1.941907101179424, 0.5222889291317159, 0.7318174570920598], [9.338437401493472, 1.8953331526329642, 0.2723665708953812, 0.579444283477627], [15.66834735944441, 2.361128624916076, 0.13517536123949858, 0.45439655223936326], [6.859926370030567, 1.9056835469948614, 0.5680997909272343, 0.7710088443153513], [13.894483626020792, 2.3569981029912044, 0.18591705611187137, 0.5003513273512102], [10.278809164263361, 2.036883333243822, 0.4285076973517382, 0.6583137325533632], [9.41752009246164, 2.3198271845516407, 0.35607317131127114, 0.6280913320768128], [6.389642277075843, 1.8230031053141544, 0.6558042048150489, 0.8347389298749421], [7.1427635929178805, 1.9063892305901176, 0.3842535533384388, 0.6561362538768046], [11.543773739497617, 1.9668775370999387, 0.48959503748778077, 0.7219922263461633], [11.913765109735653, 2.

In [15]:
print(dt_result)

[[13.191153032998058, 2.8148743062929493, 0.3652988465358318, 0.6247127308739263], [5.429111506593046, 1.9416829710144927, 0.7036405914669615, 0.855398542491366], [8.120840696435826, 2.2755535260849498, 0.4171666971261029, 0.6525146390503062], [11.926370188046583, 2.2739725718281214, 0.07071972926524317, 0.4514012307059602], [25.244469686180977, 2.8918478487886383, -0.3933849484063292, 0.24170106114015608], [10.723932277436896, 2.6195667464054018, 0.32482240437129684, 0.6233551948839655], [9.822148986885257, 1.9702522214627478, 0.4245166515165185, 0.6715234392829892], [23.36249303932974, 2.696145570657413, -0.29893305044226737, 0.4560641371107953], [11.612617536922976, 2.6918089668814047, 0.20598247628783728, 0.6110597156390574], [7.098609900566565, 2.0667332458113754, 0.6176136983131029, 0.7979221226031562], [9.231036193163897, 2.1862741968236854, 0.20423269495008145, 0.6029127018376704], [11.489324038010329, 2.282594741986933, 0.4920025169198604, 0.7047764843726882], [12.786205999739

In [16]:
print(mlp_result)

[[10.216046593791773, 2.394927204490516, 0.5084480832946849, 0.7293290564913217], [9.552738831304724, 2.26509825937055, 0.4785437678931309, 0.7711949682149895], [19.797341251440884, 2.694161021052671, -0.4208565616566178, 0.4833709475318556], [9.186939086212082, 2.0569232758844684, 0.28417103388124876, 0.6347172717471298], [14.147597666915313, 2.5580896110551903, 0.21911413112475175, 0.494923446613645], [12.917703182264209, 2.4427643105680565, 0.18670282970763408, 0.6557631787508189], [16.16680329413998, 2.943831245206599, 0.05278100480780845, 0.5012163161547768], [8.880634868317236, 1.9171858615606165, 0.5062448977535056, 0.7190927073813667], [13.177220271022717, 2.8263461196425186, 0.0990021176759116, 0.5400051306787723], [8.638724694722502, 2.0989918397678156, 0.5346511452837328, 0.7458683132890473], [9.536194396338141, 2.237253354275755, 0.17792633931758217, 0.5341424236963701], [15.883502889353885, 2.5131523465759913, 0.29771503844840497, 0.5514326645215722], [12.918649019279249, 

In [18]:
with open("rf_result.csv", "w", newline='') as csvfile:
    for i in rf_result:
        csvfile.write(str(i))
        csvfile.write('\n')

In [19]:
with open("xgb_result.csv", "w", newline='') as csvfile:
    for i in xgb_result:
        csvfile.write(str(i))
        csvfile.write('\n')

In [20]:
with open("dt_result.csv", "w", newline='') as csvfile:
    for i in dt_result:
        csvfile.write(str(i))
        csvfile.write('\n')

In [21]:
with open("mlp_result.csv", "w", newline='') as csvfile:
    for i in mlp_result:
        csvfile.write(str(i))
        csvfile.write('\n')